# Constructing a OD for Starlink in GEODYN

## What is needed to get a *GOOD* orbit determination:
 1. Spacecraft mass.
 2. Areas and reflectivities of the various spacecraft surfaces.
 3. Spacecraft Attitude Profile algorithm (I doubt there are quaternions for attitude)
    1.  If there are solar panels, what is their attitude WRT spacecraft.
 4. Spacecraft body coordinates for GPS antenna tracking point
 5. Spacecraft body coordinates for center of mass of the spacecraft
 6. RINEX data files for the GPS receiver on board
 7. Initial conditions.
 
 
 
 
## With the TLEs we have:
- Epoch
  - year
  - day of year
  
- Ballistic Coefficient  
  - Also called the first derivative of mean motion, the ballistic coefficient is the daily rate of change in the number of revs the object completes each day, divided by 2. Units are revs/day. This is "catch all" drag term used in the Simplified General Perturbations (SGP4) predictor.  
   
- Second Derivative of Mean Motion  
  - The second derivative of mean motion is a second order drag term in the SGP4 predictor used to model terminal orbit decay. It measures the second time derivative in daily mean motion, divided by 6. Units are revs/day^3. A leading decimal must be applied to this value. The last two characters define an applicable power of 10.  
  
- Drag Term ($B^*$, Radiation Pressure Coeff.)  
  - Also called the radiation pressure coefficient (or BSTAR), the parameter is another drag term in the SGP4 predictor. *Units are earth radii^-1*. The last two characters define an applicable power of 10. Do not confuse this parameter with "B-Term", the USSPACECOM special perturbations factor of drag coefficient, multiplied by reference area, divided by weight.  

- Inclination (degrees)
  - The angle between the equator and the orbit plane. The value provided is the TEME mean inclination.
  
- Right Ascension of the Ascending Node (degrees)
  - The angle between vernal equinox and the point where the orbit crosses the equatorial plane (going north). The value provided is the TEME mean right ascension of the ascending node.
  
- Eccentricity
  - A constant defining the shape of the orbit (0=circular, Less than 1=elliptical). The value provided is the mean eccentricity. A leading decimal must be applied to this value.
  
  
- Argument of Perigee (degrees)
  - The angle between the ascending node and the orbit's point of closest approach to the earth (perigee). The value provided is the TEME mean argument of perigee.
  
- Mean Anomaly (degrees)
  - The angle, measured from perigee, of the satellite location in the orbit referenced to a circular orbit with radius equal to the semi-major axis.
  
- Mean Motion (revolutions per day)
  - The value is the mean number of orbits per day the object completes. There are 8 digits after the decimal, leaving no trailing space(s) when the following element exceeds 9999.
  
  
- Revolution Number and Check Sum
    - The orbit number at Epoch Time. This time is chosen very near the time of true ascending node passage as a matter of routine. The last digit is the check sum for line 2.
    
---
    
## What does GEODYN need ?
 - Spacecraft mass
 - Areas and reflectivities of the various spacecraft surfaces.
 - Initial conditions.
     - EPOCH information
     
     
     
     
**What we use**
```
1. setup file    (fort 05)
    - 
2. gravity field (fort 12)
3. ephemerides   (fort 01)
4. gdntable      (fort 02)
5. ATGRAVFIL     (fort 18)
```
---


**Example Setup File**
```
XXXXXXXX                                                                                  
XXXXXXXX     GLOBAL SET                                                                             
XXXXXXXX                                                                                  
I32G2E         10                                                                         
SATCUT                         -10.0                                                      
EARTH  2      100100     .39860044150000D+15 .637813646D+07 .2982572D+03                  
GXPAND       1100  0 100                                         1.0D-10 1.0D-10          
OBSVU 4                                                                                   
TERMVU  1221211211                                                                        
VECOPT2        50                239000000.0                                                                                                                                   
ENDGLB                                                                                    
********************************************************************************          
********                      GEODYN ARC TITLE                          ********          
********************************************************************************          
REFSYS 111           30103000000.0000000                                                  
SATPAR           2012001 5.5000000000000D-02 100.000000D+00                               
EPOCH                30103000000.        30103000000.        30104000000.0000000          
ELEMS11                5979289.660000000   1054389.950000000   3149263.990000000          
ELEMS2                -3464.597030000000   -610.920480000000   6774.151810000000          
SATPAR           2012002 5.5000000000000D-02 100.000000D+00                               
EPOCH                30103000000.        30103000000.        30104000000.0000000          
ELEMS11                5630720.730000000    992927.110000000   3753093.260000000          
ELEMS2                -4128.340440000000   -727.965110000000   6378.401000000000                 
SOLRAD           2012001  0.                                                              
SOLRAD           2012002  0.                                                              
DRAG   0 0       2012001 0.0000000000000E+00                                              
DRAG   0 0       2012002 0.0000000000000E+00                                              
STEP             2012001      1.0                                                         
STEP             2012002      1.0                                                         
ORBTVU           2012001      30103000000.00 30104000000.00  .100000D+02                  
ORBTVU           2012002      30103000000.00 30104000000.00  .100000D+02                  
EDIT                       10000000.000        1000000000.0                               
RESIDU12                                                                                  
EMATRX                               30103.0                                              
DATA                                                                                      
PREPRO                                                                                    
SIGMA          44                1.0D-07                                                  
SELECT         44                        30103000000.00      30104000000.00               
ENDARC                                                                                    
```
- **Global set:**
   1. Mandatory Run Description on three cards with no defined format
   2. Parameter Option cards which may be used to define and/or estimate conditions which are common to all the arcs being processed
   3. Optional Station Coordinate Subgroup which may be used to alter GEODYN II’s set of tracking stations
   4. Mandatory Global Set Termination card
   
- **Arc Set:**
   1. Arc Description on three cards and  Mandatory cards to describe the reference coordinate system and time and spacecraft parameters in this arc.
       - REFSYS
       - SATPAR
       - EPOCH
       - ELEMS11
       - ELEMS2
   2. Arc Option cards which may be specified to make use of GEODYN II’s individual arc capabilities
   3. Data Selection/Deletion
       - The Data Selection/Deletion Subgroup should not be present in orbit generation runs.
       - In data reduction runs, the use of DATA and SELECT cards is mandatory (see individual card for explanation). 
   4. The last group in the Arc Set is the mandatory Arc Set Termination card


**Collected notes on ORBIT GENERATION run type**
- Orbit generation mode (ORBGEN mode) is implied by the lack of a DATA card or simulated data generation (SIMDAT) card.
- must have ORBTVU card
    - For an orbit generation run , the users should make sure that all arc parameters have zero sigmas. Orbit generation with arc parameter estimation request are incompatible in GEODYN.

- if GEODYN is performing an ORBGEN, the drag effects (Variable area model) are circumvented
- INTCOM
    - INTCOM: Turns on the intercomparison capability. This option allows the user to perform certain specific orbit generation runs. In these runs some assumptions are made , or certain perturbations only are turned on in order to study the output orbit and/or compare it with orbits generated by similar software packages. The output orbit is writen on unit59 at a rate depending on the users request on the ORBTVU card and expressed in inertial True -of -Date Cartesian elements. [NOTE 2] The assumptions and/or supressions for each individual case are described under Case Indicator on columns 9-10.
    - NOTE [3]: Sample UNIT05 setups for the orbit generation options exist in ZCDEP.TOPEX.UNIT05.DATA. The GLOBAL TITLE cards describe the individual cases.
- Must Omit DATA and SELECT and SIMDAT
    

In [34]:
from tletools import TLE

tle_string = """
Starlink
1 51104U 22005A   22024.66667824 -.00733898  17937-3 -16482-2 0  9998
2 51104  53.2186 247.5781 0054136 184.4741  34.8658 15.94242985  2195
"""

tle_lines = tle_string.strip().splitlines()

tle = TLE.from_lines(*tle_lines)
tle

TLE(name='Starlink', norad='51104', classification='U', int_desig='22005A', epoch_year=2022, epoch_day=24.66667824, dn_o2=-0.00733898, ddn_o6=0.00017937, bstar=-0.0016482, set_num=999, inc=53.2186, raan=247.5781, ecc=0.0054136, argp=184.4741, M=34.8658, n=15.94242985, rev_num=219)

In [31]:
# import numpy as np
# def E_to_TrueAnom(E, e ):
#     TrueAnom = 2*(np.arctan(np.sqrt((1+e)/(1-e))*np.tan(E/2)))
#     print('CHECK if sign of E should be Pos or Neg')
#     return TrueAnom



# def OE_to_RVrotating(mu, e, p, TrueAnom):
    
#     vr= np.sqrt(mu/p)*(e*np.sin(TrueAnom))
#     v_theta = np.sqrt(mu/p)*(1 + e*np.cos(TrueAnom))
    
#     r = (p)/(1 + e*np.cos(TrueAnom))
    
#     r_vec_rot = np.array([r, 0,0])
#     v_vec_rot = np.array([vr, v_theta,0])
    
#     print('r_vec_rotating frame =', r_vec_rot)
#     print('v_vec_rotating frame =',v_vec_rot)
#     return

In [32]:
# print('Mean Anomaly:', tle.M, 'deg')
# print('Mean Anomaly:', tle.M*(np.pi/180), 'rad')

# print('Ecc:         ', tle.ecc, 'deg')

In [33]:
# TrueAnom = E_to_TrueAnom(tle.M*(np.pi/180),tle.ecc)

# TrueAnom*(180/np.pi)